In [1]:
import numpy as np
from itertools import chain, combinations
from scipy import sparse

In [2]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 16
deg_v = 3 # w_c. Every bit is in this many checks
deg_c = 4 # w_r. Every check has this many bits in it
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), dtype=bool)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            vs = np.delete(vs, v_ind)
            cs =np.delete(cs, c_ind)

H = sparse.csc_matrix(H)

In [3]:
bit_nbhd = [H[:,i].indices for i in range(H.shape[1])]
check_nbhd = [H.tocsr()[i].indices for i in range(H.shape[0])]

In [4]:
# hx1 = sparse.kron(H, np.eye(H.shape[1], dtype=bool))
# hx2 = sparse.kron(np.eye(H.shape[0], dtype=bool), H.T)
# Hx = sparse.csr_matrix(sparse.hstack([hx1, hx2], ))

# hz1 = sparse.kron(np.eye(H.shape[1], dtype=bool), H)
# hz2 = sparse.kron(H.T, np.eye(H.shape[0], dtype=bool))
# Hz = sparse.csr_matrix(sparse.hstack([hz1, hz2]))

In [35]:
p = 0.05
p_mask = 0.1

# for i in range(1):
e = [1 if np.random.uniform() < p else 0 for i in range(H.shape[1])]
mask = [1 if np.random.uniform() < p_mask else 0 for i in range(H.shape[1])]
synd = H.dot(e) % 2

print(e)
print(synd)
print(mask)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 1 0 1 0 0 0 1 0]
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]


In [36]:
def bp(H, synd, p, T):
    b2c_messages_t0 = np.copy(H.A) * np.log((1-p)/p)
    b2c_messages_t1 = np.copy(b2c_messages_t0)

    c2b_messages_t0 = np.zeros(H.shape)
    c2b_messages_t1 = np.copy(c2b_messages_t0)

    def update_bit_to_check(bit, check):
        m = np.log((1-p)/p) + np.sum([c2b_messages_t0[nbr][bit] for nbr in bit_nbhd[bit] if nbr != check])
        b2c_messages_t1[check][bit] = m

    def update_check_to_bit(bit, check):
        m = (1-mask[check]) * (-1)**synd[check] * 2 * np.arctanh(np.prod([np.tanh(b2c_messages_t0[check][nbr]/2) for nbr in check_nbhd[check] if nbr != bit]))
        c2b_messages_t1[check][bit] = m


    for i in range(T):
        for check, nbhd in enumerate(check_nbhd):
            for bit in nbhd:
                update_check_to_bit(bit, check)
        for bit, nbhd in enumerate(bit_nbhd):
            for check in nbhd:
                update_bit_to_check(bit, check)

        b2c_messages_t0 = b2c_messages_t1
        c2b_messages_t0 = c2b_messages_t1
    # print(np.round(c2b_messages_t1, 2))


    return [np.log((1-p)/p) + np.sum([c2b_messages_t1[check][bit] for check in bit_nbhd[bit]]) for bit in range(H.shape[0])]
bp(H, synd, p, 4)


[13.328432959968906,
 12.32155048146898,
 6.384098764284023,
 8.380374304604704,
 -6.155098200067426,
 14.059557163773908,
 12.773537041708979,
 9.996590480252337,
 10.171344208894233,
 9.787400028008374,
 10.713351717509212,
 12.354370440427223]

In [32]:
bit2check = np.zeros(H.shape)

for j in range(H.shape[1]):
    for ind in H[:,j].indices:
        bit2check[ind][j] = np.log((1-p)/p)

for i in range(H.shape[0]):
    temp = 1.0
    for ind in H[i].tocsr().indices:
        bit2check[i][ind] = temp
        temp *= np.tanh(bit2check[i][ind]/2)

    temp = 1.0
    for ind in H[i].tocsr().indices[::-1]:
        bit2check[i][ind] *= temp
        # bit2check[i][ind] = (-1)**synd[i] * np.log((1+bit2check[i][ind])/(1-bit2check[i][ind]))
        bit2check[i][ind] = (-1)**synd[i] * np.arctanh(bit2check[i][ind])
        temp *= np.tanh(bit2check[i][ind]/2)

print(np.round(bit2check, 2))


[[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.01  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.   -0.01  0.    0.    0.    0.
   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.01  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.01  0.
   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.01]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.   -0.01]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.   -0.01  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.   -0.01  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.01  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.01
   0.    0.    0.    0.  ]
 [ 0.   

In [24]:
np.round(bit2check, 2)

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.01,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.01,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.01,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.01,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.01],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.01],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.01,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0. 